In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [23]:

# Read the CSV file
df = pd.read_csv('diabetes_prediction_dataset.csv')
print(df.columns)

Index(['gender', 'age', 'hypertension', 'heart_disease', 'smoking_history',
       'bmi', 'HbA1c_level', 'blood_glucose_level', 'diabetes'],
      dtype='object')


## <span style="color:red">Preprocessing</span>

In [24]:
null_counts = df.isnull().sum()
print(null_counts)

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64


### <span style="color:yellow">No columns have null values</span>

In [25]:
for i in df.columns:
    print("number of unique values in", i, ":", df[i].nunique())

number of unique values in gender : 3
number of unique values in age : 102
number of unique values in hypertension : 2
number of unique values in heart_disease : 2
number of unique values in smoking_history : 6
number of unique values in bmi : 4247
number of unique values in HbA1c_level : 18
number of unique values in blood_glucose_level : 18
number of unique values in diabetes : 2


### <span style="color:yellow">One-hot encoding for gender and smoking_history</span>

In [26]:
df_encoded = pd.get_dummies(df, columns=['gender', 'smoking_history'], drop_first=True)
print(df_encoded.head())

    age  hypertension  heart_disease    bmi  HbA1c_level  blood_glucose_level  \
0  80.0             0              1  25.19          6.6                  140   
1  54.0             0              0  27.32          6.6                   80   
2  28.0             0              0  27.32          5.7                  158   
3  36.0             0              0  23.45          5.0                  155   
4  76.0             1              1  20.14          4.8                  155   

   diabetes  gender_Male  gender_Other  smoking_history_current  \
0         0        False         False                    False   
1         0        False         False                    False   
2         0         True         False                    False   
3         0        False         False                     True   
4         0         True         False                     True   

   smoking_history_ever  smoking_history_former  smoking_history_never  \
0                 False             

### <span style="color:yellow">Normalizing the data</span>

In [27]:
boolean_columns = df_encoded.select_dtypes(include=bool).columns
numerical_columns = df_encoded.select_dtypes(include=np.number).columns
print(boolean_columns)
print(numerical_columns)

Index(['gender_Male', 'gender_Other', 'smoking_history_current',
       'smoking_history_ever', 'smoking_history_former',
       'smoking_history_never', 'smoking_history_not current'],
      dtype='object')
Index(['age', 'hypertension', 'heart_disease', 'bmi', 'HbA1c_level',
       'blood_glucose_level', 'diabetes'],
      dtype='object')


In [28]:
numerical_columns = numerical_columns.drop('diabetes')
boolean_columns = boolean_columns.insert(0, 'diabetes')
scaler = StandardScaler()
df_numerical_scaled = pd.DataFrame(scaler.fit_transform(df[numerical_columns]), columns=numerical_columns)
df_scaled = pd.concat([df_numerical_scaled, df_encoded[boolean_columns]], axis=1)

In [29]:
df_scaled.head()

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes,gender_Male,gender_Other,smoking_history_current,smoking_history_ever,smoking_history_former,smoking_history_never,smoking_history_not current
0,1.692704,-0.284439,4.936379,-0.321056,1.001706,0.047704,0,False,False,False,False,False,True,False
1,0.538006,-0.284439,-0.202578,-0.000116,1.001706,-1.426210,0,False,False,False,False,False,False,False
2,-0.616691,-0.284439,-0.202578,-0.000116,0.161108,0.489878,0,True,False,False,False,False,True,False
3,-0.261399,-0.284439,-0.202578,-0.583232,-0.492690,0.416183,0,False,False,True,False,False,False,False
4,1.515058,3.515687,4.936379,-1.081970,-0.679490,0.416183,0,True,False,True,False,False,False,False


### <span style="color:yellow">Train test split</span>

In [30]:
x = df_scaled.drop('diabetes', axis=1)
y = df_scaled['diabetes']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [31]:
df['diabetes'].value_counts()

diabetes
0    91500
1     8500
Name: count, dtype: int64

### <span style="color:yellow">Training an MLP model</span>

In [35]:
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

# Define the MLP model
mlp_model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)),  # Input layer
    Dense(32, activation='relu'),  # Hidden layer
    Dense(1)  # Output layer for regression
])

# Compile the model
mlp_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
mlp_model.fit(x_train, y_train, epochs=50, validation_split=0.2, verbose=1, callbacks=[early_stopping])

Epoch 1/50


c:\Users\ROG\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2000/2000 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0426 - val_loss: 0.0322
Epoch 2/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0312 - val_loss: 0.0308
Epoch 3/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0296 - val_loss: 0.0304
Epoch 4/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0293 - val_loss: 0.0288
Epoch 5/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0280 - val_loss: 0.0277
Epoch 6/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0270 - val_loss: 0.0270
Epoch 7/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0256 - val_loss: 0.0265
Epoch 8/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0251 - val_loss: 0.0260
Epoch 9/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0244 - val_loss: 0.0258
Epoch 10/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0251 - val_loss: 0.0253
Epoch 11/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0242 - val_loss: 0.0251
Epoch 12/50
2000/2000 ━━━━━━━━━━━━━━━━━━━

In [36]:
# Assuming x_test and y_test are already defined

# Make predictions on the test data
y_pred = mlp_model.predict(x_test)

# Calculate the Mean Squared Error (MSE) on the test data
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse}')

# Convert y_pred to binary predictions
y_pred_binary = np.where(y_pred >= 0.5, 1, 0)

# Ensure y_test is a numpy array and reshape if necessary
y_test_array = y_test.values if hasattr(y_test, 'values') else y_test
if y_test_array.ndim > 1:
    y_test_array = y_test_array.reshape(-1)

# Ensure y_pred_binary is a 1-dimensional array
if y_pred_binary.ndim > 1:
    y_pred_binary = y_pred_binary.reshape(-1)

# Compare y_pred_binary with y_test
correct_predictions = np.sum(y_pred_binary == y_test_array)
wrong_predictions = np.sum(y_pred_binary != y_test_array)

print(f"Number of Correct Predictions: {correct_predictions}")
print(f"Number of Wrong Predictions: {wrong_predictions}")

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Mean Squared Error on Test Set: 0.023293483928851316
Number of Correct Predictions: 19449
Number of Wrong Predictions: 551
